In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from scipy.spatial import distance
import pickle
import random

In [9]:
#Read the CSV File
data = pd.read_csv('data_moods.csv')

In [10]:
#Attributes and its types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              686 non-null    object 
 1   album             686 non-null    object 
 2   artist            686 non-null    object 
 3   id                686 non-null    object 
 4   release_date      686 non-null    object 
 5   popularity        686 non-null    int64  
 6   length            686 non-null    int64  
 7   danceability      686 non-null    float64
 8   acousticness      686 non-null    float64
 9   energy            686 non-null    float64
 10  instrumentalness  686 non-null    float64
 11  liveness          686 non-null    float64
 12  valence           686 non-null    float64
 13  loudness          686 non-null    float64
 14  speechiness       686 non-null    float64
 15  tempo             686 non-null    float64
 16  key               686 non-null    int64  
 1

In [11]:
data.head()

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.730,0.000000,0.0843,0.625,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.832,0.196000,0.1530,0.166,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.139,0.000077,0.0934,0.116,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.438,0.000006,0.1130,0.587,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.932,0.000000,0.1370,0.445,-3.604,0.1060,169.881,1,4,Energetic


In [12]:
#copy the original datagrame
df = data.copy()

#select the features
df = df.drop(["name","album","artist","id","release_date"],1)

<ipython-input-12-03ae0a16638c>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(["name","album","artist","id","release_date"],1)


In [13]:
col = ["length","danceability","acousticness","energy","instrumentalness",
       "liveness","valence","loudness","speechiness","tempo","key",
       "time_signature"]

#Feature Extraction
scaler = StandardScaler()
df[col] = scaler.fit_transform(df[col])

In [14]:
#One Hot Encoder
encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
enc = pd.DataFrame(encoder.fit_transform(np.array(df["mood"]).reshape(-1,1)))
enc.columns = df["mood"].unique()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
enc.head()

,Happy,Sad,Energetic,Calm
0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0


In [16]:
#dropping genre column and adding encoder matrix
df[enc.columns] = enc
df = df.drop("mood",1)
df.head()

<ipython-input-16-8aba8c34d082>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("mood",1)


,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,Happy,Sad,Energetic,Calm
0,68,2.485674,2.300744,-0.762003,0.681395,-0.852255,-0.581363,1.120746,0.446193,0.550357,-0.021123,-0.094490,0.259511,0.0,0.0,1.0,0.0
1,43,1.531242,-0.752668,-1.049750,0.994037,-0.374460,-0.097642,-0.699816,0.865853,-0.114799,0.038688,0.750992,0.259511,0.0,0.0,0.0,1.0
2,60,-0.060700,-0.973017,1.128697,-1.130088,-0.852066,-0.517290,-0.898135,-0.508493,-0.528405,0.588217,-1.503628,0.259511,0.0,0.0,0.0,1.0
3,2,0.176922,-0.217534,-0.878954,-0.213619,-0.852241,-0.379285,0.970024,-0.177803,0.165776,2.554267,-0.376318,0.259511,0.0,0.0,1.0,0.0
4,60,0.729384,-0.513431,-1.091633,1.300548,-0.852255,-0.210299,0.406800,1.062150,1.259051,1.752438,-1.221801,0.259511,0.0,1.0,0.0,0.0


In [17]:
#adding track name and artist name to the new data set
df["id"] = data["id"]
df["name"] = data["name"]
df["artist"] = data["artist"]

df_2 = df.drop(["id","artist","name"],1)

<ipython-input-17-ac127a17d362>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_2 = df.drop(["id","artist","name"],1)


In [18]:
def sim_track_find(word,artist):
    a = 0
    b = 0
    song = []
    indexes = []
    for i in data["name"]:
        if word.lower() in i.lower() and artist.lower() in data["artist"][a].lower():
            song.append(df_2[a:a+1].values)
            indexes.append(a)
            b+=1
        a+=1
    if b == 0:
        print("Nothing found. Please try something else :)")
        return 0
        
    return song[0][0], indexes[0]

In [19]:
def similar_tracks(data,song = "",artist = "",number = 5):

    if (sim_track_find(song,artist) == 0):
        return 0
    else:
        x=sim_track_find(song,artist)[0]
        index = sim_track_find(song,artist)[1]
    p = []
    count=0
    for i in df_2.values:
        p.append([distance.cosine(x,i),count])
        count+=1
    p.sort()
    song_names = df["name"]
    artist_names = df["artist"]

    print("\nSimilar songs to ",song_names[index]," by ", artist_names[index],"\n")
    for i in range(1,number+1):
        print(i,"- ",song_names[p[i][1]],", ",artist_names[p[i][1]])

In [20]:
song = "9 Crimes"
similar_tracks(df,song)


Similar songs to  9 Crimes  by  Damien Rice 

1 -  Fix You ,  Valentin
2 -  Falling Asleep ,  Tiz McNamara
3 -  Let It Go ,  James Bay
4 -  lovely (with Khalid) ,  Billie Eilish
5 -  you were good to me ,  Jeremy Zucker


In [21]:
happy = data.loc[data['mood'] == "Happy"]
happy = happy[['name','artist']]

In [22]:
sad = data.loc[data['mood'] == "Sad"]
sad = sad[['name','artist']]

In [23]:
energetic = data.loc[data['mood'] == "Energetic"]
energetic = energetic[['name','artist']]

In [24]:
calm = data.loc[data['mood'] == "Calm"]
calm = calm[['name','artist']]

In [25]:
pickle.dump(df.to_dict(), open("music_dict.pkl",'wb'))

In [26]:
pickle.dump(df_2.to_dict(), open("music_dict2.pkl",'wb'))

In [27]:
pickle.dump(happy.to_dict(), open("music_dict_happy.pkl",'wb'))

In [28]:
pickle.dump(sad.to_dict(), open("music_dict_sad.pkl",'wb'))

In [29]:
pickle.dump(energetic.to_dict(), open("music_dict_energetic.pkl",'wb'))

In [30]:
pickle.dump(calm.to_dict(), open("music_dict_calm.pkl",'wb'))